# ML TFIDF WITHOUT CLASS 0

In [31]:
#from frugal.data_preproc.data import load_and_split_data
from frugal.evaluation import evaluate, concatenate_evaluations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from datasets import load_dataset
from frugal.config.environment import DATA_DIR
import os
import pandas as pd


In [32]:
def load_datasets():
    """
    Loads train and test datasets from Hugging Face and returns them as pandas DataFrames.
    """
    train_dataset = pd.DataFrame(load_dataset('QuotaClimat/frugalaichallenge-text-train', split='train', cache_dir=DATA_DIR))
    test_dataset = pd.DataFrame(load_dataset('QuotaClimat/frugalaichallenge-text-train', split='test', cache_dir=DATA_DIR))

    print("✅ Train and test datasets successfully loaded from Hugging Face 🤗")
    return train_dataset, test_dataset


In [17]:
def make_train_test_split(train_dataset, test_dataset):
    """
    Splits datasets into features (quotes) and labels, returning separate DataFrames/Series.
    """
    X_train = train_dataset[['quote']]
    X_test = test_dataset[['quote']]
    y_train = train_dataset['label']
    y_test = test_dataset['label']

    print("✅ Data successfully split into X and y for train and test sets")
    return X_train, X_test, y_train, y_test

In [67]:
def load_and_split_data_withou_class_n(filtered_class : str):
    """
    Loads datasets from source withou a certain class "filter_class" and splits them into features and labels for training and testing.
    """
    train_dataset, test_dataset = load_datasets()

    train_dataset_filtered = train_dataset[train_dataset['label']!=filtered_class]
    test_dataset_filtered = test_dataset[test_dataset['label']!=filtered_class]
    X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = make_train_test_split(train_dataset_filtered,test_dataset_filtered)
    return X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered


In [68]:
load_and_split_data_withou_class_n("0_not_relevant")[3].value_counts()

✅ Train and test datasets successfully loaded from Hugging Face 🤗
✅ Data successfully split into X and y for train and test sets


label
5_science_unreliable               160
4_solutions_harmful_unnecessary    160
1_not_happening                    154
6_proponents_biased                139
2_not_human                        137
3_not_bad                           97
7_fossil_fuels_needed               65
Name: count, dtype: int64

In [14]:

""" # Get the directory of the current script
script_directory = os.path.dirname(os.path.abspath(__file__))
# Get the name of the folder
model_name = os.path.basename(script_directory)
print(model_name) """

# Load datasets and make data lists
X_train, X_test, y_train, y_test = load_and_split_data()
# Train and predict with model
""" y_pred = dummy_model(X_train, X_test, y_train)
# Get model performance
evaluate(model_name, y_test, y_pred)
# Concatenate all evaluations
concatenate_evaluations() """

✅ Train and test datasets successfully loaded from Hugging Face 🤗
✅ Data successfully split into X and y for train and test sets


' y_pred = dummy_model(X_train, X_test, y_train)\n# Get model performance\nevaluate(model_name, y_test, y_pred)\n# Concatenate all evaluations\nconcatenate_evaluations() '

In [15]:
X_train

,quote
0,Interesting to note that Oklahoma minimum temp...
1,REPORT ON GEO ENGINEERING A.k.a. man-made FAKE...
2,"To realize, yes, climate change is a real prob..."
3,The fluctuations in the carbon dioxide concent...
4,I know the comments that were made after that ...
...,...
4867,The morally superior choice is for people to b...
4868,The climate is changing. No one is denying cli...
4869,It will be up to 15 years before it is possibl...
4870,"Global warming is real, but a problem, not the..."


In [ ]:
def tfidf(X_train, X_test, y_train) :
    # Vectorisation
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    # Training
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    # Prediction
    y_pred = model.predict(X_test_vec)
    return y_pred